### 인스타그램 이미지 크롤러 
---
#### 2020-05-07 (목) 서동원
---
- 한울코드 + 존잘코드 퓨전!
- 최하단 'Main cell' 아래에 있는 셀만 실행하시면 됩니다.
- tqdm 구현은 못함...
- 현재 노트북 파일이 있는 위치에 download 폴더와 그 하위에 검색어별 폴더가 자동 생성됩니다.

In [45]:
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import requests
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import time
import tqdm
import os
import shutil
import re
from getpass import getpass
import math

import selenium.common.exceptions as sce
from queue import Queue
from multiprocessing.pool import ThreadPool

In [2]:
def make_dir(dirname):
    current_path = os.getcwd()
    path = os.path.join(current_path, dirname)
    if not os.path.exists(path):
        os.makedirs(path)

In [3]:
def wait_and_click(driver, xpath, thread_num):
    try:
        wait = WebDriverWait(driver, 10)
        elem = wait.until(EC.element_to_be_clickable((By.XPATH, xpath)))
        elem.click()
    except sce.TimeoutException as e:
        print(f'[thread {thread_num}: Loading timeout.. Browser refreshing\n{e}\n')
        driver.refresh()
        time.sleep(2.5)
        return wait_and_click(driver, xpath, thread_num)
    
    return 

In [4]:
def get_scroll(driver):
    pos = driver.execute_script("return window.pageYOffset;")
    return pos

In [5]:
def wait_until_elem_present(driver, xpath, thread_num):
    
    try:
        wait = WebDriverWait(driver, 10)
        elem = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        status = 1
    except sce.TimeoutException:
        print(f'[thread {thread_num}: Loading timeout..')
        status = 0
    return status

In [6]:
def collect_img_url(driver, keyword, thread_num):
    
    print('*' * 30
      + f'\n[thread {thread_num}: {keyword}]\nCollecting image links start !!\n'
      + '*' * 30, '\n')
    
    srcs = []

    count=0
    src_duplicated_patience = 0
    
    xpath_popup = '//div[@class="PdwC2 fXiEu s2MYR"]'
    xpath_videos = '//video[@class="tWeCl"]'
    xpath_imgs = '//div[@class="_2dDPU CkGkG"]//img[@class="FFVAD"]'
    
    while 1:
        try:
            status = wait_until_elem_present(driver, xpath_popup, thread_num)   
            imgs = driver.find_elements(By.XPATH, xpath_imgs)

            for img in imgs:
                src = img.get_attribute('src')

                if src not in srcs and src is not None:
                    srcs.append(src)
                    print(f'[thread {thread_num}: {keyword}]\n{count}: {src}\n')
                    count += 1

                if src in srcs:
                    src_duplicated_patience += 1
                    if src_duplicated_patience > 10:
                        break
        except sce.StaleElementReferenceException as e:
            print(e)                        

        # 다음 사진으로 넘기기
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.RIGHT)
    
    links = list(set(srcs))
    print('*' * 30
      + f'\n[thread {thread_num}: {keyword}]\nCollecting image links success : {len(links)} links\n'
      + '*' * 30, '\n')
    
    return links  # 중복 제거 후 반환

In [7]:
def collect_img_url_2(driver, keyword, thread_num):
    
    print('*' * 30
      + f'\n[thread {thread_num}: {keyword}]\nCollecting image links start !!\n'
      + '*' * 30, '\n')
    
    xpath_box =  '//div[@class="block col-lg-4 col-md-6 col-sm-12"]'
    xpath_img = '//img[@class="lazy loaded"]'

    boxes = driver.find_elements(By.XPATH, xpath_box)
    print(len(boxes))

    elems = [box.find_element(By.XPATH, xpath_img) for box in boxes]
    print(len(elems))

    links = [elem.get_attribute('src') for elem in elems]
    print(len(links))
    
    links = list(set(links))
    
    print('*' * 30
      + f'\n[thread {thread_num}: {keyword}]\nCollecting image links success : {len(links)} links\n'
      + '*' * 30, '\n')
    
    return links

In [8]:
def get_ext(src):
    ext = src.split('.')[-1].lower()
    ext = ext.split('?', 1)[0]
    
    if ext == 'jpg' or ext == 'jpeg':
        ext = 'jpg'
    elif ext == 'png':
        ext = 'png'
    elif ext == 'gif':
        ext = 'gif'
    else:
        ext = 'jpg'
    
    return ext

In [9]:
def save_response_to_file(response, file_path):
    
    with open(file_path, 'wb') as file:
        shutil.copyfileobj(response.raw, file)
    del response


In [10]:
def all_files(dir_path):
    paths = []
    for root, dirs, files in os.walk(dir_path):
        for file in files:
            if os.path.isfile(dir_path + '/' + file):
                paths.append(dir_path + '/' + file)
    return paths

In [11]:
def validate_img(dir_path):
    paths = all_files(dir_path)
    
    invalid_paths = list(filter(lambda x: len(x.split('.')[-1]) > 4, paths))
    for path in invalid_paths:
        os.remove(path)
        print(f'Invalid file deleted: {path}')
    
    jpeg_paths = list(filter(lambda x: len(x.split('.')[-1]) == 'jpeg', paths))
    for path in jpeg_paths:
        new_path = path.split('.')[0] + 'jpg'
        os.rename(path, new_path)
        print('File name renamed:')
        print(f"'{path}'\nto\n'{new_path}'\n")

In [12]:
def login(driver, login_type, account_info, thread_num):
    if login_type == 0:
        login_with_instagram_account(driver, account_info, thread_num)
    elif login_type == 1:
        login_with_facebook_account(driver, account_info, thread_num)
    else:
        print('Check login type\n')

In [13]:
def login_with_instagram_account(driver, account_info, thread_num):
    
    print(f'[thread {thread_num}: Login with instagram account\n')
    
    # 입력상자 elem 찾기 
#     xpath_input_box = '//div[@class="EPjEi"]//input[@class="_2hvTZ pexuQ zyHYP"]'
#     wait_and_click(driver, xpath_input_box)
#     input_box = driver.find_elements(By.XPATH, xpath_input_box)
    
    
    # 아이디 입력 창을 찾아 변수에 저장
    xpath_id = """/html/body/div[5]/div/div[2]/div[2]/div/div/div[1]/div/form/div[2]/div/label/input"""
#    xpath_id = '//div[@class="_9GP1n   "]//input[@class="_2hvTZ pexuQ zyHYP"]'
    wait_and_click(driver, xpath_id, thread_num)
    input_id = driver.find_element(By.XPATH, xpath_id)
    # 아이디 입력 창에 아이디를 입력
    id_ = account_info[0]
    input_id.send_keys(id_)
    
    # 비밀번호 입력 창을 찾아 변수에 저장 
    xpath_pw = """/html/body/div[5]/div/div[2]/div[2]/div/div/div[1]/div/form/div[3]/div/label/input"""
    input_pw = driver.find_element(By.XPATH, xpath_pw)
    # 비밀번호 입력 창에 비밀번호를 입력
    # getpass 모듈 getpass 함수로 입력값 마스킹 하여 받기
    password = account_info[1]
    input_pw.send_keys(password)

    # 엔터키 입력하여 로그인
    xpath_submit = """/html/body/div[5]/div/div[2]/div[2]/div/div/div[1]/div/form/div[4]"""
    driver.find_element(By.XPATH, xpath_submit).submit()
    
    time.sleep(7)
    
#     # 화면 뜰때까지 대기
#     xpath_first_img = '//div[@class="FFVAD"]'
#     wait_until_elem_present(driver, xpath_first_img, thread_num) 

In [14]:
def login_with_facebook_account(driver, account_info, thread_num):
    
    print(f'[thread {thread_num}: Login with facebook account\n')
    
    xpath_fb = '//span[@class="KPnG0"]'
    wait_and_click(driver, xpath_fb, thread_num)

    time.sleep(1)

    # 아이디 입력 창을 찾아 변수에 저장
    input_id = driver.find_element_by_id('email')
    # 아이디 입력 창에 아이디를 입력
    id_ = account_info[0]
    input_id.send_keys(id_)
    
    
    # 비밀번호 입력 창을 찾아 변수에 저장
    input_pw = driver.find_element_by_id('pass')
    # 비밀번호 입력 창에 비밀번호를 입력
    password = account_info[1]
    input_pw.send_keys(password)
    
    # 엔터키 입력하여 로그인
    input_pw.send_keys(Keys.ENTER)


In [15]:
def check_request(req, keyword, thread_num):
    if req.status_code == 200:
        print('*' * 30
          + f'\n[thread {thread_num}: {keyword}]\nGet URL request sucess !!\n'
          + '*' * 30, '\n')
        status = 1
    else:
        html = BeautifulSoup(req.text, 'html.parser')
        text = html.find(class_ = 'error-container -cx-PRIVATE-ErrorPage__errorContainer -cx-PRIVATE-ErrorPage__errorContainer__').text
        text = text.replace('\n', '')
        text = re.sub('\s{2,}', ' ', text)
        print('*' * 100
            + f'\n[thread {thread_num}: {keyword}]\nGet URL request fail !!\n'
            + f'[{req.status_code} Error]\n{text}\n'
            + f"높은 확률로 해당 검색어 검색결과 없음. 인스타그램에서 '{keyword}' 검색결과 확인 요망\n"
            + '*' * 100, '\n')
        
        status = 0
       
    return status

In [16]:
def scroll_down_to_end(driver, keyword, thread_num):
    
    last_scroll = 0
    scroll_patient = 0

    print('*' * 30
          + f'\n[thread {thread_num}: {keyword}]\nScrolling down..\n'
          + '*' * 30, '\n')

    while 1:

        scroll = get_scroll(driver)
        if scroll == last_scroll:
            scroll_patient += 1
        else:
            scroll_patient = 0
            last_scroll = scroll

        if scroll_patient > 5:
            break

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        WebDriverWait(driver, 10).until(lambda driver: driver.execute_script('return document.readyState') == 'complete')
        time.sleep(0.5)



In [17]:
def saving_imgs(driver, keyword, links, dir_path, thread_num):
    
    print('*' * 30
          + f'\n[thread {thread_num}: {keyword}]\nSaving image start !!\n'
          + '*' * 30, '\n')
    
    for i, link in enumerate(links):
        
        try:
            response = requests.get(link, stream=True)
            print(f'[thread {thread_num}: {keyword}]\n{i}: image downloaded\n')
        except Exception as e:
            print(f'[thread {thread_num}: {keyword}]\n{i}: download failed\n', e)
            continue
        try:    
            ext = get_ext(link)  # 파일 확장자
            file_without_ext = f'instagram_{keyword}_{str(i).zfill(4)}'  # 확장자 제외한 파일명
            file_path = f'{dir_path}/{file_without_ext}.{ext}'  # 파일 전체 경로
            
            # 이미지 파일로 저장
            save_response_to_file(response, file_path) 
            print(f'[thread {thread_num}: {keyword}]\n{i}: image saved\n')
        except Exception as e:
            print(f'[thread {thread_num}: {keyword}]\n{i}: save failed\n', e)
    
    print('*' * 30
          + f'\n[thread {thread_num}: {keyword}]\nSaving image file success !!\n'
          + '*' * 30, '\n')    

In [18]:
def do_instagram_search(driver_path, keyword, thread_num):
    
    os.makedirs('./download', exist_ok=True)
    os.makedirs('./download/instagram', exist_ok=True)
    
    driver = Chrome(driver_path)
    session = requests.Session()

    # 요청 보낼 때 header와 함께 전송
    header = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
               AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.92 Safari/537.36", 
              "Accept":"text/html,application/xhtml+xml,application/xml;\
              q=0.9,imgwebp,*/*;q=0.8"}
    
    # 검색 url
#     naver_img_search =  'https://search.naver.com/search.naver?where=image&sm=tab_jum&query=' + keyword
#     daum_img_search = 'https://search.daum.net/search?w=img&nil_search=btn&DA=NTB&enc=utf8&q=' + keyword
#     insta_img_search = 'https://www.instagram.com/explore/tags/' + keyword
    insta_img_search = 'https://imgtagram.com/tag/' + keyword
    
    # 세션 get
    req = session.get(insta_img_search, headers=header)
    status = check_request(req,  keyword, thread_num)
    if status == 0:
        # 세션 및 드라이버 종료
        session.close()
        driver.quit()
        return
    else:
        pass
   
    # 드라이버 get
    driver.get(insta_img_search)
    time.sleep(1)  
    
    # 스크롤 최하단까지 내리기
    scroll_down_to_end(driver, keyword, thread_num)
    
    # 이미지 링크 수집
    links = collect_img_url_2(driver, keyword, thread_num)
    
#     # 세션 및 드라이버 종료
#     session.close()
#     driver.quit()
    
    # 디렉토리 생성
    dir_path = f'./download/instagram/{keyword}'
    make_dir(dir_path)
    
    # 이미지 저장
    saving_imgs(driver, keyword, links, dir_path, thread_num)
    
    # 이미지 유효성 체크
    validate_img(dir_path)

In [19]:
def do_crawling(driver_path, keywords):
    # 동시 처리할 스레드 개수 설정
    n_thread = len(keywords)
    tasks = []
    for thread_num, keyword in enumerate(keywords):
        tasks.append([driver_path, keyword, thread_num])

    with ThreadPool(n_thread) as tp:
        # 스레드에 입력할 매개변수가 2개 이상일 경우 starmap을 사용한다.
        tp.starmap(do_instagram_search, tasks)
    
    
    # 종료 전 메세지
    print()
    print('=' * 60)
    
    # 폴더별 사진 수 카운트
    keyword_dirs = list(os.walk('./download/instagram'))[0][1]
    for dir in keyword_dirs:
        files= []
        path = './download/instagram/' + dir
        for _, _, files in os.walk(path):
            for file in files:
                if os.path.isfile(file):
                    files.append(file)
        print(f'{dir} : {len(files)}')
    
    print('\nAll crawling tasks success')
    print('End program')

In [20]:
def get_id_pw(login_type):
    if login_type == 0:
        print('You select instagram account login')
    elif login_type == 1:
        print('You select instagram account login')
    else:
        print('Invalid login type. Check the right code')

    id_ = input('id or email : ')
    password = getpass('password : ')
    print()
    
    return id_, password

---
## main cell
- Parameters
    - driver_path : 본인의 Chrome driver 경로
    - daum_keywords : 검색어 리스트 (6개 이하로 추천) 

In [ ]:
# main

# 크롬 드라이버 경로
driver_path = 'c:/chromedriver.exe'

# 검색어 리스트 (6개 이하 추천)
instagram_keywords = \
[
'트위스트펌'
,'남자 트위스트펌'
,'여자 트위스트펌'
,'트위스트펌 연예인'
]

instagram_keywords = list(map(lambda x: re.sub(' ', '', x), instagram_keywords))
do_crawling(driver_path, instagram_keywords)

******************************
[thread 2: 여자트위스트펌]
Get URL request sucess !!
****************************** 

******************************
[thread 1: 남자트위스트펌]
Get URL request sucess !!
****************************** 

******************************
[thread 0: 트위스트펌]
Get URL request sucess !!
****************************** 

******************************
[thread 3: 트위스트펌연예인]
Get URL request sucess !!
****************************** 

******************************
[thread 2: 여자트위스트펌]
Scrolling down..
****************************** 

******************************
[thread 1: 남자트위스트펌]
Scrolling down..
****************************** 

******************************
[thread 0: 트위스트펌]
Scrolling down..
****************************** 

******************************
[thread 여자트위스트펌: 2]
****************************** 

******************************
[thread 여자트위스트펌: 2]
****************************** 

******************************
[thread 2: 여자트위스트펌]
Saving image start !!
*******************


[thread 0: 트위스트펌]
61: image downloaded

[thread 0: 트위스트펌]
61: image saved

[thread 0: 트위스트펌]
62: image downloaded

[thread 0: 트위스트펌]
62: image saved

[thread 0: 트위스트펌]
63: image downloaded

[thread 0: 트위스트펌]
63: image saved

[thread 0: 트위스트펌]
64: image downloaded

[thread 0: 트위스트펌]
64: image saved

[thread 0: 트위스트펌]
65: image downloaded

[thread 0: 트위스트펌]
65: image saved

[thread 0: 트위스트펌]
66: image downloaded

[thread 0: 트위스트펌]
66: image saved

[thread 0: 트위스트펌]
67: image downloaded

[thread 0: 트위스트펌]
67: image saved

[thread 0: 트위스트펌]
68: image downloaded

[thread 0: 트위스트펌]
68: image saved

[thread 0: 트위스트펌]
69: image downloaded

[thread 0: 트위스트펌]
69: image saved

[thread 0: 트위스트펌]
70: image downloaded

[thread 0: 트위스트펌]
70: image saved

[thread 0: 트위스트펌]
71: image downloaded

[thread 0: 트위스트펌]
71: image saved

[thread 0: 트위스트펌]
72: image downloaded

[thread 0: 트위스트펌]
72: image saved

[thread 0: 트위스트펌]
73: image downloaded

[thread 0: 트위스트펌]
73: image saved

[thread 0: 트위스트펌]
74: im


[thread 0: 트위스트펌]
168: image saved

[thread 0: 트위스트펌]
169: image downloaded

[thread 0: 트위스트펌]
169: image saved

[thread 0: 트위스트펌]
170: image downloaded

[thread 0: 트위스트펌]
170: image saved

[thread 0: 트위스트펌]
171: image downloaded

[thread 0: 트위스트펌]
171: image saved

[thread 0: 트위스트펌]
172: image downloaded

[thread 0: 트위스트펌]
172: image saved

[thread 0: 트위스트펌]
173: image downloaded

[thread 0: 트위스트펌]
173: image saved

[thread 0: 트위스트펌]
174: image downloaded

[thread 0: 트위스트펌]
174: image saved

[thread 0: 트위스트펌]
175: image downloaded

[thread 0: 트위스트펌]
175: image saved

[thread 0: 트위스트펌]
176: image downloaded

[thread 0: 트위스트펌]
176: image saved

[thread 0: 트위스트펌]
177: image downloaded

[thread 0: 트위스트펌]
177: image saved

[thread 0: 트위스트펌]
178: image downloaded

[thread 0: 트위스트펌]
178: image saved

[thread 0: 트위스트펌]
179: image downloaded

[thread 0: 트위스트펌]
179: image saved

[thread 0: 트위스트펌]
180: image downloaded

[thread 0: 트위스트펌]
180: image saved

[thread 0: 트위스트펌]
181: image downloaded


[thread 0: 트위스트펌]
275: image downloaded

[thread 0: 트위스트펌]
275: image saved

[thread 0: 트위스트펌]
276: image downloaded

[thread 0: 트위스트펌]
276: image saved

[thread 0: 트위스트펌]
277: image downloaded

[thread 0: 트위스트펌]
277: image saved

[thread 0: 트위스트펌]
278: image downloaded

[thread 0: 트위스트펌]
278: image saved

[thread 0: 트위스트펌]
279: image downloaded

[thread 0: 트위스트펌]
279: image saved

[thread 0: 트위스트펌]
280: image downloaded

[thread 0: 트위스트펌]
280: image saved

[thread 0: 트위스트펌]
281: image downloaded

[thread 0: 트위스트펌]
281: image saved

[thread 0: 트위스트펌]
282: image downloaded

[thread 0: 트위스트펌]
282: image saved

[thread 0: 트위스트펌]
283: image downloaded

[thread 0: 트위스트펌]
283: image saved

[thread 0: 트위스트펌]
284: image downloaded

[thread 0: 트위스트펌]
284: image saved

[thread 0: 트위스트펌]
285: image downloaded

[thread 0: 트위스트펌]
285: image saved

[thread 0: 트위스트펌]
286: image downloaded

[thread 0: 트위스트펌]
286: image saved

[thread 0: 트위스트펌]
287: image downloaded

[thread 0: 트위스트펌]
287: image saved


[thread 0: 트위스트펌]
381: image saved

[thread 0: 트위스트펌]
382: image downloaded

[thread 0: 트위스트펌]
382: image saved

[thread 0: 트위스트펌]
383: image downloaded

[thread 0: 트위스트펌]
383: image saved

[thread 0: 트위스트펌]
384: image downloaded

[thread 0: 트위스트펌]
384: image saved

[thread 0: 트위스트펌]
385: image downloaded

[thread 0: 트위스트펌]
385: image saved

[thread 0: 트위스트펌]
386: image downloaded

[thread 0: 트위스트펌]
386: image saved

[thread 0: 트위스트펌]
387: image downloaded

[thread 0: 트위스트펌]
387: image saved

[thread 0: 트위스트펌]
388: image downloaded

[thread 0: 트위스트펌]
388: image saved

[thread 0: 트위스트펌]
389: image downloaded

[thread 0: 트위스트펌]
389: image saved

[thread 0: 트위스트펌]
390: image downloaded

[thread 0: 트위스트펌]
390: image saved

[thread 0: 트위스트펌]
391: image downloaded

[thread 0: 트위스트펌]
391: image saved

[thread 0: 트위스트펌]
392: image downloaded

[thread 0: 트위스트펌]
392: image saved

[thread 0: 트위스트펌]
393: image downloaded

[thread 0: 트위스트펌]
393: image saved

[thread 0: 트위스트펌]
394: image downloaded


[thread 0: 트위스트펌]
488: image downloaded

[thread 0: 트위스트펌]
488: image saved

[thread 0: 트위스트펌]
489: image downloaded

[thread 0: 트위스트펌]
489: image saved

[thread 0: 트위스트펌]
490: image downloaded

[thread 0: 트위스트펌]
490: image saved

[thread 0: 트위스트펌]
491: image downloaded

[thread 0: 트위스트펌]
491: image saved

[thread 0: 트위스트펌]
492: image downloaded

[thread 0: 트위스트펌]
492: image saved

[thread 0: 트위스트펌]
493: image downloaded



In [64]:
###################################################################################################################################
###################################################################################################################################

# 아래부터 실험중인 것

In [28]:
def scroll_down_n_search(driver, keyword, thread_num):
 
    print('*' * 30
          + f'\n[thread {thread_num}: {keyword}]\nScrolling down..\n'
          + '*' * 30, '\n')
    
    last_scroll = 0
    scroll_patient = 0
    srcs = []
    
    while 1:
        
        imgs = driver.find_elements(By.XPATH, '//img[@class="lazy loaded"]')
        
        for img in imgs:
            src = img.get_attribute('src')
            if src not in srcs and src is not None:
                srcs.append(src)
        
        scroll = get_scroll(driver)
        if scroll == last_scroll:
            scroll_patient += 1
        else:
            scroll_patient = 0
            last_scroll = scroll

        if scroll_patient > 5:
            break

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        WebDriverWait(driver, 10).until(lambda driver: driver.execute_script('return document.readyState') == 'complete')
        time.sleep(0.5)
    
    links = list(set(srcs)) 
    return links

In [32]:
options = webdriver.ChromeOptions()

options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('User-Agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36')
options.add_argument("Accept=text/html,application/xhtml+xml,application/xml;q=0.9,imgwebp,*/*;q=0.8")
options.add_argument('disable-notifications')
options.add_argument('disable-infobars')
prefs = {"profile.default_content_setting_values.notifications": 2}

options.add_experimental_option("prefs", prefs)

keyword = '남자트위스트펌'
driver = Chrome('c:/chromedriver', options=options)
driver.get('https://imgtagram.com/tag/' + keyword)


In [33]:
links = scroll_down_to_end(driver, keyword, 0)

******************************
[thread 0: 남자트위스트펌]
Scrolling down..
****************************** 



In [37]:
atags = driver.find_element(By.XPATH, '//div[@class="photo-section col-lg-12"]').find_elements(By.TAG_NAME, 'a')
print('atags :', len(atags))

srcs = [atag.get_attribute('href') for atag in atags]
print('srcs :', len(srcs))

srcs = list(filter(lambda x: 'https://imgtagram.com/m/' in x, hrefs))
print('posts', len(srcs))

srcs  = list(set(srcs))
print('srcs :', len(srcs))

atags : 216
srcs : 216
posts 126
srcs : 63


In [63]:
interval = np.linspace(0, len(srcs)-1, 5)
interval = list(map(lambda x: math.ceil(x), interval))
interval

[0, 16, 31, 47, 62]

In [52]:
def get_img_links_with_thread(driver_path, srcs, thread_num):
    
    n_thread = 4
    
    interval = np.linspace(0, len(srcs)-1, 5)
    interval = list(map(lambda x: math.ceil(x), interval))

    tasks = []
    
    for i in range(n_thread):
        tasks.append([
              driver_path
            , srcs[interval[i]:interval[i + 1]]
            , f'{thread_num}-{i}'
        ])

    with ThreadPool(n_thread) as tp:
        # 스레드에 입력할 매개변수가 2개 이상일 경우 starmap을 사용한다.
        results = tp.starmap(get_img_links, tasks)
    
    return results

In [58]:
def get_img_links(driver_path, links, thread_num):
    
    driver_sub = Chrome(driver_path)
    session = requests.Session()

    # 요청 보낼 때 header와 함께 전송
    header = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
               AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.92 Safari/537.36", 
              "Accept":"text/html,application/xhtml+xml,application/xml;\
              q=0.9,imgwebp,*/*;q=0.8"}
    
    img_links = []
    
    for link in links:
    
        # 세션 get
        req = session.get(link, headers=header)
        status = check_request(req,  keyword, thread_num)
        if status == 0:
            # 세션 및 드라이버 종료
            session.close()
            driver_sub.quit()
            return
        else:
            pass

        # 드라이버 get
        driver_sub.get(link)
        xpath_img = '//img[@class="lazy loaded"]'
        wait_until_elem_present(driver_sub, xpath_img, thread_num)
        src = driver.find_element(By.XPATH, xpath_img).get_attribute('src')
        
        if src not in img_links and src is not None:
            img_links.append(src)
    
    return img_links

In [59]:
driver_path = 'c:/chromedriver.exe'
thread_num = 0

results = get_img_links_with_thread(driver_path, srcs, thread_num)

******************************
[thread 0-1: 남자트위스트펌]
Get URL request sucess !!
************************************************************
[thread 0-2: 남자트위스트펌]
Get URL request sucess !!
******************************  



******************************
[thread 0-0: 남자트위스트펌]
Get URL request sucess !!
****************************** 

******************************
[thread 0-3: 남자트위스트펌]
Get URL request sucess !!
****************************** 

******************************
[thread 0-0: 남자트위스트펌]
Get URL request sucess !!
****************************** 

******************************
[thread 0-1: 남자트위스트펌]
Get URL request sucess !!
****************************** 

******************************
[thread 0-2: 남자트위스트펌]
Get URL request sucess !!
****************************** 

******************************
[thread 0-3: 남자트위스트펌]
Get URL request sucess !!
****************************** 

******************************
[thread 0-0: 남자트위스트펌]
Get URL request sucess !!
*************************

In [62]:
results

[['https://scontent-lax3-1.cdninstagram.com/v/t51.2885-15/e35/93879326_103631084597568_3517584910123175406_n.jpg?_nc_ht=scontent-lax3-1.cdninstagram.com&_nc_cat=100&_nc_ohc=0pxv1kVz5BAAX-1gMi9&oh=d24c225fb07745173d5a380416f0ee9d&oe=5EDC78F1&ig_cache_key=MjI4Nzg1NzY1NTY5MDk5Nzc1MA%3D%3D.2'],
 ['https://scontent-lax3-1.cdninstagram.com/v/t51.2885-15/e35/93879326_103631084597568_3517584910123175406_n.jpg?_nc_ht=scontent-lax3-1.cdninstagram.com&_nc_cat=100&_nc_ohc=0pxv1kVz5BAAX-1gMi9&oh=d24c225fb07745173d5a380416f0ee9d&oe=5EDC78F1&ig_cache_key=MjI4Nzg1NzY1NTY5MDk5Nzc1MA%3D%3D.2'],
 ['https://scontent-lax3-1.cdninstagram.com/v/t51.2885-15/e35/93879326_103631084597568_3517584910123175406_n.jpg?_nc_ht=scontent-lax3-1.cdninstagram.com&_nc_cat=100&_nc_ohc=0pxv1kVz5BAAX-1gMi9&oh=d24c225fb07745173d5a380416f0ee9d&oe=5EDC78F1&ig_cache_key=MjI4Nzg1NzY1NTY5MDk5Nzc1MA%3D%3D.2'],
 ['https://scontent-lax3-1.cdninstagram.com/v/t51.2885-15/e35/93879326_103631084597568_3517584910123175406_n.jpg?_nc_ht=scon

In [ ]:
#  실험 셀 마지막

###################################################################################################################################
###################################################################################################################################

In [31]:
len(links)

9

In [25]:
collect_img_url_2(driver, keyword, 0)

******************************
[thread 0: 남자트위스트펌]
****************************** 

74
74
74
******************************
[thread 0: 남자트위스트펌]
****************************** 



['https://scontent-lax3-1.cdninstagram.com/v/t51.2885-15/e35/93879326_103631084597568_3517584910123175406_n.jpg?_nc_ht=scontent-lax3-1.cdninstagram.com&_nc_cat=100&_nc_ohc=0pxv1kVz5BAAX-1gMi9&oh=d24c225fb07745173d5a380416f0ee9d&oe=5EDC78F1&ig_cache_key=MjI4Nzg1NzY1NTY5MDk5Nzc1MA%3D%3D.2']

In [26]:
xpath_box =  '//div[@class="block col-lg-4 col-md-6 col-sm-12"]'
#xpath_box = """//*[@id="tag"]/div/div[3]/div/div/div[38]"""
xpath_img = '//img[@class="lazy loaded"]'

boxes = driver.find_elements(By.XPATH, xpath_box)
print(len(boxes))

elems = [box.find_element(By.XPATH, xpath_img) for box in boxes]
print(len(elems))

links = [elem.get_attribute('data-src') for elem in elems]
print(len(links))

74
74
74


In [55]:
main = driver.find_elements(By.XPATH, '//div[@class="photo-section col-lg-12"]')
main

[<selenium.webdriver.remote.webelement.WebElement (session="758665ce920714485f90ce284715609f", element="b39d5583-66f7-4965-8005-051bdf06e52d")>]

In [56]:
boxes = main[0].find_elements(By.XPATH, xpath_box)

In [64]:
boxes[0].find_element(By.XPATH, '//img[@class="lazy loaded"]').

<selenium.webdriver.remote.webelement.WebElement (session="758665ce920714485f90ce284715609f", element="d493958d-faa5-4efc-bd23-49473329cec0")>

In [65]:
boxes[1].find_element(By.XPATH, '//img[@class="lazy loaded"]')

<selenium.webdriver.remote.webelement.WebElement (session="758665ce920714485f90ce284715609f", element="d493958d-faa5-4efc-bd23-49473329cec0")>

In [52]:
boxes[0].find_element(By.XPATH, xpath_img).get_attribute('src')

'https://scontent-lax3-1.cdninstagram.com/v/t51.2885-15/e35/93879326_103631084597568_3517584910123175406_n.jpg?_nc_ht=scontent-lax3-1.cdninstagram.com&_nc_cat=100&_nc_ohc=0pxv1kVz5BAAX-1gMi9&oh=d24c225fb07745173d5a380416f0ee9d&oe=5EDC78F1&ig_cache_key=MjI4Nzg1NzY1NTY5MDk5Nzc1MA%3D%3D.2'

In [53]:
boxes[2].find_element(By.XPATH, xpath_img).get_attribute('src')

'https://scontent-lax3-1.cdninstagram.com/v/t51.2885-15/e35/93879326_103631084597568_3517584910123175406_n.jpg?_nc_ht=scontent-lax3-1.cdninstagram.com&_nc_cat=100&_nc_ohc=0pxv1kVz5BAAX-1gMi9&oh=d24c225fb07745173d5a380416f0ee9d&oe=5EDC78F1&ig_cache_key=MjI4Nzg1NzY1NTY5MDk5Nzc1MA%3D%3D.2'

In [33]:
links

['https://scontent-lax3-1.cdninstagram.com/v/t51.2885-15/e35/93879326_103631084597568_3517584910123175406_n.jpg?_nc_ht=scontent-lax3-1.cdninstagram.com&_nc_cat=100&_nc_ohc=0pxv1kVz5BAAX-1gMi9&oh=d24c225fb07745173d5a380416f0ee9d&oe=5EDC78F1&ig_cache_key=MjI4Nzg1NzY1NTY5MDk5Nzc1MA%3D%3D.2',
 'https://scontent-lax3-1.cdninstagram.com/v/t51.2885-15/e35/93879326_103631084597568_3517584910123175406_n.jpg?_nc_ht=scontent-lax3-1.cdninstagram.com&_nc_cat=100&_nc_ohc=0pxv1kVz5BAAX-1gMi9&oh=d24c225fb07745173d5a380416f0ee9d&oe=5EDC78F1&ig_cache_key=MjI4Nzg1NzY1NTY5MDk5Nzc1MA%3D%3D.2',
 'https://scontent-lax3-1.cdninstagram.com/v/t51.2885-15/e35/93879326_103631084597568_3517584910123175406_n.jpg?_nc_ht=scontent-lax3-1.cdninstagram.com&_nc_cat=100&_nc_ohc=0pxv1kVz5BAAX-1gMi9&oh=d24c225fb07745173d5a380416f0ee9d&oe=5EDC78F1&ig_cache_key=MjI4Nzg1NzY1NTY5MDk5Nzc1MA%3D%3D.2',
 'https://scontent-lax3-1.cdninstagram.com/v/t51.2885-15/e35/93879326_103631084597568_3517584910123175406_n.jpg?_nc_ht=scontent-la

In [93]:
set(links)

{'https://scontent-lax3-1.cdninstagram.com/v/t51.2885-15/e35/93879326_103631084597568_3517584910123175406_n.jpg?_nc_ht=scontent-lax3-1.cdninstagram.com&_nc_cat=100&_nc_ohc=0pxv1kVz5BAAX-1gMi9&oh=d24c225fb07745173d5a380416f0ee9d&oe=5EDC78F1&ig_cache_key=MjI4Nzg1NzY1NTY5MDk5Nzc1MA%3D%3D.2'}

In [72]:
atags = driver.find_element(By.XPATH, '//div[@class="photo-section col-lg-12"]').find_elements(By.TAG_NAME, 'a')
len(atags)

216

In [73]:
hrefs = [atag.get_attribute('href') for atag in atags]
len(hrefs)

216

In [74]:
hrefs

['https://imgtagram.com/m/2287857658358428254',
 'https://imgtagram.com/m/2287857658358428254',
 'https://imgtagram.com/m/2282880675823297253',
 'https://imgtagram.com/m/2282880675823297253',
 'https://imgtagram.com/m/2278234387979786902',
 'https://imgtagram.com/m/2278234387979786902',
 'https://imgtagram.com/m/2278233588663023355',
 'https://imgtagram.com/m/2278233588663023355',
 'https://imgtagram.com/m/2245435991153455372',
 'https://imgtagram.com/tag/Repost',
 'https://imgtagram.com/u/lloydbomb_hyunjeong',
 'https://imgtagram.com/u/get_repost',
 'https://imgtagram.com/m/2245435991153455372',
 'https://imgtagram.com/m/2238978737888666338',
 'https://imgtagram.com/tag/%EC%8A%A4%ED%95%80%EC%8A%A4%EC%99%88%EB%A1%9C%ED%8E%8C',
 'https://imgtagram.com/tag/%EB%82%A8%EC%9E%90%ED%9E%88%ED%94%BC%ED%8E%8C',
 'https://imgtagram.com/m/2238978737888666338',
 'https://imgtagram.com/m/2237622039651637719',
 'https://imgtagram.com/tag/%ED%8A%B8%EC%9C%84%EC%8A%A4%ED%8A%B8%ED%8E%8C',
 'https://imgta

In [77]:
real_hrefs = list(filter(lambda x: 'https://imgtagram.com/m/' in x, hrefs))
len(real_hrefs)

126

In [81]:
real_without_dup  = list(set(real_hrefs))
len(real_without_dup)

63

In [82]:
real_without_dup

['https://imgtagram.com/m/2091365033645558709',
 'https://imgtagram.com/m/2015087155474817519',
 'https://imgtagram.com/m/2023192295727091214',
 'https://imgtagram.com/m/1856538824776110779',
 'https://imgtagram.com/m/1592792329337279017',
 'https://imgtagram.com/m/1979776493697625691',
 'https://imgtagram.com/m/2282880675823297253',
 'https://imgtagram.com/m/2028986166796973627',
 'https://imgtagram.com/m/2023193529338990190',
 'https://imgtagram.com/m/1852634966480291487',
 'https://imgtagram.com/m/1836184173901255286',
 'https://imgtagram.com/m/1861667894535385373',
 'https://imgtagram.com/m/2127546281407713298',
 'https://imgtagram.com/m/2028796413178366143',
 'https://imgtagram.com/m/1662939112791857779',
 'https://imgtagram.com/m/1486975408469877127',
 'https://imgtagram.com/m/1822365785316454051',
 'https://imgtagram.com/m/1480164190031041005',
 'https://imgtagram.com/m/2033261635200001406',
 'https://imgtagram.com/m/2245435991153455372',
 'https://imgtagram.com/m/21754416113330

In [68]:
driver.find_elements(By.XPATH, '//img[@class="lazy loaded"]')

[<selenium.webdriver.remote.webelement.WebElement (session="758665ce920714485f90ce284715609f", element="8d9f33cc-84ca-4b6e-8cc7-827aa7a6d0ba")>,
 <selenium.webdriver.remote.webelement.WebElement (session="758665ce920714485f90ce284715609f", element="4e6a73d2-3a9f-4ccb-93a6-5050cddd440b")>,
 <selenium.webdriver.remote.webelement.WebElement (session="758665ce920714485f90ce284715609f", element="a2c37842-3cbe-4e80-856b-1e1015ab15ae")>,
 <selenium.webdriver.remote.webelement.WebElement (session="758665ce920714485f90ce284715609f", element="27822bee-f5a7-458e-8f03-3af93e128623")>,
 <selenium.webdriver.remote.webelement.WebElement (session="758665ce920714485f90ce284715609f", element="02e8f3ac-0fe6-4d2e-a138-c5d3879a6599")>,
 <selenium.webdriver.remote.webelement.WebElement (session="758665ce920714485f90ce284715609f", element="cffbb717-c503-414d-9554-b1f6d8ca137c")>,
 <selenium.webdriver.remote.webelement.WebElement (session="758665ce920714485f90ce284715609f", element="48c1f748-ab2e-41e0-8603-3b

In [91]:
driver.find_elements(By.XPATH, '//img[@class="lazy loaded"]')[0].get_attribute('src')

'https://scontent-lax3-1.cdninstagram.com/v/t51.2885-15/e35/93879326_103631084597568_3517584910123175406_n.jpg?_nc_ht=scontent-lax3-1.cdninstagram.com&_nc_cat=100&_nc_ohc=0pxv1kVz5BAAX-1gMi9&oh=d24c225fb07745173d5a380416f0ee9d&oe=5EDC78F1&ig_cache_key=MjI4Nzg1NzY1NTY5MDk5Nzc1MA%3D%3D.2'